While most datasets are sampled at 25 'frames' per second, the 'Utrecht' dataset is sampled at a different rate, so it needed to be resampled. In the end the 'Utrecht' dataset was not used in the thesis because of its small size and large number of issues.

In [2]:
import collections, os
import pandas as pd

In [3]:
Recording = collections.namedtuple('Recording',['fn','df']) # filename and dataframe 
def load_data(dirpath):
    dfs = []
    for root, _, filenames in os.walk(dirpath):
        for filename in filenames:
            filepath = os.path.join(root,filename)
            nxt = pd.read_csv(filepath,index_col=0,compression='gzip')
            dfs.append(Recording(filename,nxt))
    return dfs

In [41]:
# load all files from data_clean directory into a list of DataFrames
data = load_data('../data_clean')

In [42]:
data_utrecht = [rec for rec in data if rec.fn.startswith('utrecht')]

In [65]:
def resample(rec):
    df = rec.df.copy()
    df.index = pd.TimedeltaIndex(df['Trial time (s)'],'s')
    return Recording(rec.fn,df.resample('40ms').interpolate()[:45001])

In [66]:
data_utrecht_resampled = list(map(resample,data_utrecht))

In [67]:
[len(df) for fn,df in data_utrecht_resampled]

[45001, 45001, 45001, 45001, 45001, 45001, 45001, 45001, 45001, 45001, 45001]

In [69]:
for fn,df in data_utrecht_resampled:
    filepath = os.path.join('..','data_clean',fn.split('.')[0]+'_resampled'+'.csv.gz')
    df.to_csv(filepath,compression='gzip')